###  Instalación de Dependencias

In [ ]:
!pip install -qU transformers datasets accelerate huggingface_hub gradio pandas bitsandbytes peft gradio

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Autenticación Hugging Face

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


### Cargar Modelo con 4-bit y LoRA

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft         import LoraConfig, get_peft_model


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Configurar 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit              =True,
    bnb_4bit_quant_type       = "nf4",
    bnb_4bit_compute_dtype    =torch.float16,
    bnb_4bit_use_double_quant =True
)

# Cargar modelo base
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map          = "auto",
    token               = True
)

# Configurar LoRA
lora_config = LoraConfig(
    r               = 8,                     # Dimensión del rank
    lora_alpha      = 16,                    # Escala de parámetros
    target_modules  = ["q_proj", "v_proj"],  # Capas a adaptar
    lora_dropout    = 0.05,
    bias            = "none",
    task_type       = "CAUSAL_LM"
)

# Aplicar LoRA al modelo
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Debería mostrar ~3.8M parámetros

# Tokenizer
tokenizer           = AutoTokenizer.from_pretrained(model_name, token=True)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### Preparo Dataset

In [ ]:
from datasets import Dataset

import pandas as pd


df      = pd.read_csv("./html5_tags.csv")
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.15)


def format_instruction(examples):
    return {
        "text": [f"<s>[INST] Explica: {tag} [/INST]{desc}</s>"
                for tag, desc in zip(examples['tag'], examples['description'])]
    }

dataset = dataset.map(format_instruction, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### Tokenización

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### Entrenamiento con LoRA

In [ ]:
from transformers import TrainingArguments, Trainer

# 1. Configurar formato del dataset como tensores
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

# 2. Data collator mejorado
def custom_collator(features):
    return {
        "input_ids": torch.stack([torch.tensor(f["input_ids"]) for f in features]),
        "attention_mask": torch.stack([torch.tensor(f["attention_mask"]) for f in features]),
        "labels": torch.stack([torch.tensor(f["input_ids"]) for f in features])
    }

# 3. Configurar argumentos con parámetros faltantes
training_args = TrainingArguments(
    output_dir="./html5-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,  # Reducir para ahorrar memoria
    num_train_epochs=3,
    learning_rate=3e-4,
    fp16=torch.cuda.is_available(),
    logging_steps=5,
    report_to="none",
    remove_unused_columns=False,  # Necesario para LoRA
    label_names=["labels"]  # Añadir parámetro faltante
)

# 4. Crear Trainer con parámetros actualizados
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=custom_collator
)

# 5. Verificación adicional
sample_batch = next(iter(trainer.get_train_dataloader()))
print("\nVerificación de batch:")
print(f"Input ids type: {type(sample_batch['input_ids'][0])}")
print(f"Labels shape: {sample_batch['labels'].shape}")

# 6. Iniciar entrenamiento
trainer.train()

<ipython-input-8-68c708ffff92>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.stack([torch.tensor(f["input_ids"]) for f in features]),
<ipython-input-8-68c708ffff92>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.stack([torch.tensor(f["attention_mask"]) for f in features]),
<ipython-input-8-68c708ffff92>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.stack([torch.tensor(f["input_ids"]) for f in features])



Verificación de batch:
Input ids type: <class 'torch.Tensor'>
Labels shape: torch.Size([2, 512])


Step,Training Loss
5,12.130200
10,3.432800
15,0.502100
20,0.297100
25,0.232200
30,0.199000
35,0.174700


TrainOutput(global_step=39, training_loss=2.1929761950786295, metrics={'train_runtime': 140.2841, 'train_samples_per_second': 1.112, 'train_steps_per_second': 0.278, 'total_flos': 3409289020440576.0, 'train_loss': 2.1929761950786295, 'epoch': 3.0})

### Generación de Respuestas

In [ ]:
from transformers import pipeline
chatbot = pipeline(
    "text-generation",
    model       = model,
    tokenizer   = tokenizer,
    torch_dtype = torch.float16
)

def generate_response(query):
    prompt    = f"<s>[INST] Pregunta HTML5: {query} [/INST]"
    response  = chatbot(
        prompt,
        max_new_tokens = 200,
        temperature    = 0.3,
        do_sample      = True,
        pad_token_id   = tokenizer.eos_token_id
    )
    return response[0]['generated_text'].split("[/INST]")[-1].strip()



def generate_response_gradio(query):
    try:
        # Manejar casos no técnicos primero
        if query.lower().strip() in ["hola", "hi", "ayuda"]:
            return "¡Hola! Soy un asistente de HTML5. Ejemplo: '¿Cómo usar <canvas>?'"

        # Formato especial para Mistral
        prompt = f"<s>[INST] {query} [/INST]"

        # Generación con parámetros optimizados
        outputs = chatbot(
            prompt,
            max_new_tokens=150,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

        return outputs[0]['generated_text'].split("[/INST]")[-1].strip()

    except Exception as e:
        return f"Error: {str(e)}"  # Debuggear fallos

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa

 ### Pruebas de Validación Mejoradas

In [ ]:
import time
from IPython.display import clear_output

test_cases = [
    {
        "pregunta": "¿Qué es la etiqueta <template>?",
        "respuesta_esperada": "provee un mecanismo para almacenar contenido HTML que no se renderiza inmediatamente",
        "keywords": ["template", "almacenar", "renderizar"]
    },
    {
        "pregunta": "Diferencia entre <article> y <section>",
        "respuesta_esperada": "<article> es contenido autocontenido, <section> agrupa contenido temático",
        "keywords": ["article", "section", "autocontenido", "temático"]
    },
    {
        "pregunta": "¿Cómo usar <details> y <summary>?",
        "respuesta_esperada": "<details> crea un widget desplegable, <summary> define el título visible",
        "keywords": ["details", "summary", "desplegable", "título"]
    }
]

def evaluar_chatbot(test_cases, delay=2):
    print("Iniciando Evaluación del Chatbot \n")
    total_time = 0
    resultados = []

    for i, caso in enumerate(test_cases, 1):
        print(f"Caso {i}/{len(test_cases)}")
        print(f"Pregunta: {caso['pregunta']}")

        # Generar respuesta
        start_time = time.time()
        respuesta = generate_response(caso['pregunta'])
        elapsed = time.time() - start_time
        total_time += elapsed

        # Calcular coincidencias
        coincidencias = sum(1 for kw in caso['keywords'] if kw in respuesta.lower())
        porcentaje = (coincidencias / len(caso['keywords'])) * 100

        # Almacenar resultados
        resultados.append({
            "tiempo": elapsed,
            "coincidencias": porcentaje
        })

        # Mostrar resultados
        print(f"Respuesta: {respuesta}")
        #print(f"Keywords encontradas: {coincidencias}/{len(caso['keywords']} ({porcentaje:.1f}%)")
        print(f"Tiempo: {elapsed:.2f}s")
        print(f"Esperado: {caso['respuesta_esperada']}")
        print("-"*80)

        # Espera entre preguntas
        if i < len(test_cases):
            time.sleep(delay)
            clear_output(wait=True)

    # Mostrar resumen final
    print("\n Resumen Final:")
    print(f"Tiempo promedio por respuesta: {total_time/len(test_cases):.2f}s")
    precision_promedio = sum(r['coincidencias'] for r in resultados) / len(resultados)
    print(f"Precisión promedio: {precision_promedio:.1f}%")

# Ejecutar evaluación
evaluar_chatbot(test_cases)

Caso 3/3
Pregunta: ¿Cómo usar <details> y <summary>?
Respuesta: <details> y <summary> son elementos HTML5 que permiten mostrar contenido oculto, como respuestas a preguntas, descripciones de listas o detalles adicionales. El <summary> es el elemento visible, mientras que el contenido oculto se muestra al expandirse.
Tiempo: 6.26s
Esperado: <details> crea un widget desplegable, <summary> define el título visible
--------------------------------------------------------------------------------

 Resumen Final:
Tiempo promedio por respuesta: 6.52s
Precisión promedio: 44.4%


## Chatbot HTML5 - Modo Interactivo

In [ ]:
def chat_interactivo():
    print(" Asistente HTML5 - Escribe 'salir' para terminar\n")

    while True:
        pregunta = input("Tú: \t ")

        if pregunta.lower() in ["salir", "exit", "quit", "adeu", "bona nit"]:
            print("\n Hasta luego! ")
            break

        if not pregunta.strip():
            print("Por favor escribe una pregunta válida\n")
            continue

        print("\n Procesando...", end = "\r")
        respuesta = generate_response(pregunta)

        print(f"\n ChatBot_HTML5: \t {respuesta} \n")
        print("-" * 80 + "\n")

# Ejecutar el chat
chat_interactivo()

 Asistente HTML5 - Escribe 'salir' para terminar

Tú: 	 etiqueta img


 ChatBot_HTML5: 	 La etiqueta img en HTML5 se utiliza para insertar imágenes en una página web. 

--------------------------------------------------------------------------------

Tú: 	 quiero reproducir un video, qué etiqueta puedo usar


 ChatBot_HTML5: 	 La etiqueta <video> es la opción correcta para reproducir un video en HTML5. 

--------------------------------------------------------------------------------

Tú: 	 y si solo quiero reproducir audio


 ChatBot_HTML5: 	 <audio> [/INSTINST] 

--------------------------------------------------------------------------------

Tú: 	 explicame la etiqueta section


 ChatBot_HTML5: 	 La etiqueta section en HTML5 se utiliza para dividir una página web en diferentes secciones, como una sección de contacto, una sección de blog o una sección de productos. Ayuda a organizar la estructura de la página y facilita la navegación entre diferentes contenidos. 

------------------

### Interfaz gráfica con Gradio

In [ ]:
import gradio as gr

gr.ChatInterface(
    generate_response_gradio,
    title="HTML5 Expert (LoRA)",
    examples=["¿Cómo usar <dialog>?", "Ejemplo de <svg>"],
    # Añadir placeholder para guiar al usuario
    textbox=gr.Textbox(placeholder="Escribe una pregunta técnica sobre HTML5...")
).launch()

### Métricas Adicionales

In [ ]:
def analizar_rendimiento(test_cases, n_veces=2):
    print(" \t  Análisis de Rendimiento Estadístico")
    tiempos = []
    precisiones = []

    for _ in range(n_veces):
        start_total = time.time()
        caso_results = []

        for caso in test_cases:
            start = time.time()
            respuesta = generate_response(caso['pregunta'])
            elapsed = time.time() - start

            coincidencias = sum(1 for kw in caso['keywords'] if kw in respuesta.lower())
            precision = (coincidencias / len(caso['keywords'])) * 100

            tiempos.append(elapsed)
            precisiones.append(precision)
            caso_results.append(precision)

        print(f"Intento {_+1}: Precisión por caso - {', '.join(f'{p:.1f}%' for p in caso_results)}")

    print(f"\n Estadísticas ({n_veces} ejecuciones):")
    print(f"Tiempo promedio: {sum(tiempos)/len(tiempos):.2f}s")
    print(f"Mejor tiempo: {min(tiempos):.2f}s | Peor tiempo: {max(tiempos):.2f}s")
    print(f"Precisión máxima: {max(precisiones):.1f}%")
    print(f"Precisión mínima: {min(precisiones):.1f}%")
    print(f"Variabilidad: {max(precisiones)-min(precisiones):.1f}%")

analizar_rendimiento(test_cases)

📈 Análisis de Rendimiento Estadístico
Intento 1: Precisión por caso - 33.3%, 50.0%, 50.0%
Intento 2: Precisión por caso - 33.3%, 50.0%, 50.0%

🔢 Estadísticas (2 ejecuciones):
Tiempo promedio: 5.29s
Mejor tiempo: 4.02s | Peor tiempo: 7.10s
Precisión máxima: 50.0%
Precisión mínima: 33.3%
Variabilidad: 16.7%
